# SETUP

## is GPU on?

In [ ]:
# from tensorflow.python.client import device_lib
# print(device_lib.list_local_devices())

## boilerplate

In [1]:
from tensorflow.keras.callbacks import Callback, EarlyStopping
from numpy import arange

MODEL_NAME = 'mobilenet'
IMG_SIZE = (224, 224)
INPUT_SHAPE=(224, 224, 3)
CLASSES = 2
FT_BLOCK = 10 # FROM feature extractor TO fine tuning scratch
BATCH_SIZE = 64 # these data points will be passed as a batch at one time to the network
PR_RATIO = list(arange(1,10,1))
TRIAL = list(arange(0,10,1))
MODEL_DIR = '../../models/PR'

## UDC/Fs

In [2]:
from tensorflow.keras.applications.mobilenet import MobileNet
from tensorflow.keras import layers
from math import floor
from timeit import default_timer as timer
from json import dump
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow import data
from keras.engine import training

def model_constructor(FT_BLOCK):
    base_model = MobileNet(
        weights='imagenet', 
        include_top=False,
        input_shape=INPUT_SHAPE)
    
    # construct the top layer containing 
    x = base_model.output
    x = layers.GlobalAveragePooling2D(keepdims=True)(x)
    x = layers.Dropout(1e-3, name='dropout')(x)
    x = layers.Conv2D(CLASSES, (1, 1), padding='same', name='conv_preds')(x)
    x = layers.Reshape((CLASSES,), name='reshape_2')(x)
    x = layers.Activation(activation='sigmoid', name='predictions')(x)
    
    # model is ready to be trained
    alpha, rows = 1.0, 224
    model = training.Model(inputs=base_model.input, outputs=x, name='mobilenet_%0.2f_%s' % (alpha, rows))

    # freeze the layers before the `FROZEN_END` layer 
    total_layers = len(base_model.layers)
    ft_layers = floor(total_layers / 10)
    FROZEN_END = total_layers - ft_layers * FT_BLOCK

    base_model.trainable = True
    for layer in base_model.layers[:FROZEN_END]: 
        layer.trainable = False
    
    return model

class TimeCallback(Callback):
    def __init__(self, logs={}):
        self.logs=[]
    def on_epoch_begin(self, epoch, logs={}):
        self.starttime = timer()
    def on_epoch_end(self, epoch, logs={}):
        self.logs.append(timer()-self.starttime)

def time_converter(sec):
    hours, rem = divmod(sec, 3600)
    minutes, seconds = divmod(rem, 60)
    print("{:0>2}:{:0>2}:{:05.2f}".format(int(hours),int(minutes),seconds))

def save_history(history, tag):
    file_path = f'{HISTORY_DIR}/{tag}.json'
    with open(file_path, 'w') as f:
        dump(history.history, f)
        
def data_preparation(BATCH_SIZE):
    train_dir = '../../data/binaryclass_clean/train/'
    test_dir = '../../data/binaryclass_clean/test/'

    train_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='training')

    val_ds = image_dataset_from_directory(
        directory=train_dir,
        label_mode='categorical',
        batch_size=BATCH_SIZE,
        image_size=IMG_SIZE,
        seed=0,
        validation_split=0.1,
        subset='validation')

    test_ds = image_dataset_from_directory(
        directory=test_dir,
        label_mode='categorical',
        batch_size=1,
        image_size=IMG_SIZE)

    AUTOTUNE = data.AUTOTUNE
    train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
    val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
    
    return train_ds, val_ds, test_ds

import tempfile
from os.path import getsize

def get_gzipped_model_size(file):
  # Returns size of gzipped model, in Mbytes.
  import os
  import zipfile

  zipped_file = f'{MODEL_DIR}/{tag}.zip'
  with zipfile.ZipFile(zipped_file, 'w', compression=zipfile.ZIP_DEFLATED) as f:
    f.write(file)
    zipped_size = getsize(zipped_file) / float(2**20)
#     remove(zipped_file)

  return zipped_size

In [3]:
train_ds, val_ds, test_ds = data_preparation(BATCH_SIZE)

Found 9980 files belonging to 2 classes.
Using 8982 files for training.
Found 9980 files belonging to 2 classes.
Using 998 files for validation.
Found 1000 files belonging to 2 classes.


# GRID SEARCH

In [4]:
import tensorflow_model_optimization as tfmot
from tensorflow.keras.models import save_model

for i in TRIAL:
    print(" - - - - - TRIAL:", i, " - - - - - ")
    loss, accuracy, model_size = [None]*len(PR_RATIO), [None]*len(PR_RATIO), [None]*len(PR_RATIO)

    for pr in PR_RATIO:
        tag = f'{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}_PR{pr*10}_{i}'
        print(tag)

        logdir = f'../../logs/PR/{tag}.json'
        callbacks = [
            tfmot.sparsity.keras.UpdatePruningStep(),
            tfmot.sparsity.keras.PruningSummaries(log_dir=logdir),
        ]
        pruning_params = {
            'pruning_schedule': tfmot.sparsity.keras.ConstantSparsity(
                target_sparsity=pr/10, begin_step=0, end_step=-1, frequency=1)}

        model = model_constructor(FT_BLOCK)
        prune_low_magnitude = tfmot.sparsity.keras.prune_low_magnitude
        model_for_pruning = prune_low_magnitude(model, **pruning_params)
        # model_for_pruning.summary()

        model_for_pruning.compile(loss='binary_crossentropy', metrics='accuracy', optimizer = 'adam')
        history = model_for_pruning.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=callbacks)

        loss[pr-1], accuracy[pr-1] = model_for_pruning.evaluate(test_ds)

    # PRINT THE RESULT
    print(f' * * * * * {tag[:-2]}_10-90_LOSS * * * * * ')
    for item in loss:
        print(item)
    print(f' * * * * * {tag[:-2]}_10-90_ACCURACY * * * * * ')
    for item in accuracy:
        print(item)
    print(f' * * * * * {tag[:-2]}_10-90_MODEL SIZE * * * * * ')
    for item in model_size:
        print(item)

 - - - - - TRIAL: 0  - - - - - 
mobilenet_BS64_FT100_PR10_0


/usr/local/lib/python3.6/dist-packages/keras/engine/base_layer.py:2223: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  warnings.warn('`layer.add_variable` is deprecated and '


1000/1000 [==============================] - 6s 6ms/step - loss: 0.5237 - accuracy: 0.9010
mobilenet_BS64_FT100_PR20_0
1000/1000 [==============================] - 7s 7ms/step - loss: 0.8402 - accuracy: 0.8900
mobilenet_BS64_FT100_PR30_0
1000/1000 [==============================] - 7s 6ms/step - loss: 0.6329 - accuracy: 0.9140
mobilenet_BS64_FT100_PR40_0
1000/1000 [==============================] - 6s 6ms/step - loss: 0.7873 - accuracy: 0.8990
mobilenet_BS64_FT100_PR50_0
1000/1000 [==============================] - 7s 6ms/step - loss: 0.9492 - accuracy: 0.8800
mobilenet_BS64_FT100_PR60_0
1000/1000 [==============================] - 6s 6ms/step - loss: 0.7319 - accuracy: 0.8780
mobilenet_BS64_FT100_PR70_0
1000/1000 [==============================] - 7s 6ms/step - loss: 0.7485 - accuracy: 0.8790
mobilenet_BS64_FT100_PR80_0
1000/1000 [==============================] - 6s 6ms/step - loss: 1.1767 - accuracy: 0.8400
mobilenet_BS64_FT100_PR90_0
1000/1000 [==============================] - 7s 

1000/1000 [==============================] - 6s 6ms/step - loss: 0.8613 - accuracy: 0.9080
mobilenet_BS64_FT100_PR20_5
1000/1000 [==============================] - 6s 6ms/step - loss: 0.6290 - accuracy: 0.8970
mobilenet_BS64_FT100_PR30_5
1000/1000 [==============================] - 7s 7ms/step - loss: 0.7005 - accuracy: 0.8960
mobilenet_BS64_FT100_PR40_5
1000/1000 [==============================] - 6s 6ms/step - loss: 0.8123 - accuracy: 0.8980
mobilenet_BS64_FT100_PR50_5
1000/1000 [==============================] - 6s 6ms/step - loss: 0.8507 - accuracy: 0.8870
mobilenet_BS64_FT100_PR60_5
1000/1000 [==============================] - 6s 6ms/step - loss: 0.8853 - accuracy: 0.8790
mobilenet_BS64_FT100_PR70_5
1000/1000 [==============================] - 6s 6ms/step - loss: 0.7628 - accuracy: 0.8220
mobilenet_BS64_FT100_PR80_5
1000/1000 [==============================] - 6s 6ms/step - loss: 1.5436 - accuracy: 0.7020
mobilenet_BS64_FT100_PR90_5
1000/1000 [==============================] - 7s 

1000/1000 [==============================] - 11s 11ms/step - loss: 0.9000 - accuracy: 0.8450
mobilenet_BS64_FT100_PR80_7
1000/1000 [==============================] - 10s 10ms/step - loss: 0.9198 - accuracy: 0.8950
mobilenet_BS64_FT100_PR90_7
1000/1000 [==============================] - 10s 10ms/step - loss: 0.8893 - accuracy: 0.7370
 * * * * * mobilenet_BS64_FT100_PR90_10-90_LOSS * * * * * 
0.8884993195533752
0.7635126709938049
0.8302448987960815
0.9818260073661804
0.6035231351852417
0.9017170667648315
0.8999611139297485
0.9197598099708557
0.8893251419067383
 * * * * * mobilenet_BS64_FT100_PR90_10-90_ACCURACY * * * * * 
0.8859999775886536
0.890999972820282
0.9089999794960022
0.8399999737739563
0.875
0.859000027179718
0.8450000286102295
0.8949999809265137
0.7369999885559082
 * * * * * mobilenet_BS64_FT100_PR90_10-90_MODEL SIZE * * * * * 
None
None
None
None
None
None
None
None
None
 - - - - - TRIAL: 8  - - - - - 
mobilenet_BS64_FT100_PR10_8
1000/1000 [==============================] - 1

# MODEL WITHOUT PRUNE

In [9]:
model = model_constructor(FT_BLOCK)
model.compile(loss='binary_crossentropy', metrics='accuracy', optimizer='adam')
model.fit(train_ds, epochs=100, verbose=0, validation_data=val_ds, callbacks=[])
model.evaluate(test_ds)

1000/1000 [==============================] - 9s 9ms/step - loss: 0.7638 - accuracy: 0.8830


[0.7637704014778137, 0.8830000162124634]

In [10]:
# Save the model
model_path = f'{MODEL_DIR}/{MODEL_NAME}_BS{BATCH_SIZE}_FT{FT_BLOCK*10}.h5'
model.save(model_path)

In [11]:
print("Model size in Mb:", getsize(model_path) / float(2**20))

Model size in Mb: 37.2322998046875
